In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

At the first block of every epoch
- Get the list of elected validators (elected_list)
- Get the stakes of validators who were eligible for election in the last epoch (last block)
- Check if any of these validators not present in elected_list

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":"1"}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getAllValidator():
    method = 'hmy_getAllValidatorAddresses'
    params = []
    return get_information(method, params)['result']
    

In [5]:
def getValidatorByBlock(validator, block):
    method = "hmy_getValidatorInformationByBlockNumber"
    params = [validator, block]
    return get_information(method, params)['result']

In [6]:
def getAllElectedValidator():
    method = "hmy_getElectedValidatorAddresses"
    params =[]
    return get_information(method, params)['result']

In [7]:
def E3_test():
    # get the last block in current epoch
    flag = True
    block = getBlockNumber()
    while block % 38 != 37:
        time.sleep(8)
        block = getBlockNumber()
    print("last block number in the last epoch is " + str(block))
    validators = getAllValidator()

    # get the first block in next epoch
    block = getBlockNumber()
    while block % 38 != 0:
        time.sleep(8)
        block = getBlockNumber()
    print("first block number in the current epoch is " + str(block))
    elected = getAllElectedValidator()
    
    hex_block = hex(block-1)
    eligible = []
    for i in validators:
        status = getValidatorByBlock(i, hex_block)['epos-status']
        if status == "currently elected and signing enough blocks to be eligible for election next epoch"\
        or status == "eligible to be elected next epoch":
            eligible.append(i)
    
    for i in eligible:
        if i not in elected:
            print("[FAILED] Validator " + i + " joined before election was not considered as the validator for election")
            flag = False
    if flag:
        return True
    return False
    

In [8]:
E3_test()

last block number in the last epoch is 26637
first block number in the current epoch is 26638
[FAILED] Validator one164e2dwupqxd7ssr85ncnkx3quk7fexy0eta2vy joined before election was not considered as the validator for election
[FAILED] Validator one12mx7ssutgvd243g5vpvefl5sf6vttqkq5z3x04 joined before election was not considered as the validator for election


False